In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq


groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(groq_api_key=groq_api_key,model_name="qwen-2.5-32b")

In [3]:
parser = StrOutputParser()

In [4]:
prompt1 = PromptTemplate(
    template="Classify the sentiment of the following feedback text into positive or negative \n{feedback}",
    input_variables=['feedback']
)

In [5]:
classifier_chain = prompt1 | model | parser
classifier_chain.invoke("This is a terrible smartphone")

'The sentiment of the feedback text "This is a terrible smartphone" is negative.'

## ***Apply PydanticOutputParser: Use structured output parser***

In [8]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.output_parsers import PydanticOutputParser

## *My schema output class*

In [9]:
class Feedback(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')
   
   
parser2 = PydanticOutputParser(pydantic_object=Feedback)

In [10]:
prompt2 = PromptTemplate(
    template="Classify the sentiment of the following feedback text into positive or negative \n{feedback} \n{format_instruction}",
    input_variables=['feedback'],
    partial_variables={
        'format_instruction': parser2.get_format_instructions()
    }
)

In [12]:
classifier_chain = prompt2 | model | parser2
response = classifier_chain.invoke("This is a terrible smartphone")
response

Feedback(sentiment='negative')

In [13]:
response.sentiment

'negative'

In [14]:
classifier_chain.invoke("This is a good smartphone").sentiment

'positive'

## ***Now Build the conditional Branch***

`branch_chain = RunnableBranch(
    ('condition', 'chain1'),
    ('condition', 'chain2'),
    'default'
)`

In [15]:
from langchain.schema.runnable import RunnableBranch, RunnableLambda

In [17]:
prompt_pos = PromptTemplate(
    template="Write the appropriate response to this positive feedback\n{feedback}",
    input_variables=['feedback']
)

prompt_neg = PromptTemplate(
    template="Write the appropriate response to this negative feedback\n{feedback}",
    input_variables=['feedback']
)

In [18]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt_pos | model | parser),
    (lambda x:x.sentiment == 'negative', prompt_neg | model | parser),
    RunnableLambda(lambda x: "Could not find any king of sentiment"),
)

In [ ]:
final_chain = classifier_chain | branch_chain

In [31]:
response = final_chain.invoke(
    {
        'feedback': "This is a terrible phone"
    }
)
print(response)

To effectively respond to negative feedback, it's important to show empathy, understanding, and a willingness to address the issues raised. Here's a general approach you could take:

---

Thank you very much for sharing your feedback with us. We truly appreciate you taking the time to let us know about your experience. We're sorry to hear that you've had a negative experience and we want to assure you that we are here to help resolve the issues you've mentioned.

Could you please provide us with more details about the specific problems you encountered? This will help us to address your concerns more effectively. We are committed to improving and ensuring that all our users have a positive experience.

Thank you again for bringing this to our attention. We value your input and look forward to making things right for you.

---

This response acknowledges the user's feelings and indicates a willingness to take action, which can help in turning a negative experience into a more positive on

In [25]:
final_chain.invoke(
    {
        'feedback': "This is a good phone and grate camera."
    }
)

"That's wonderful to hear! If there's anything else you need or if you want to share more about what you enjoyed, feel free to let me know. Thank you for your positive feedback!"

In [32]:
final_chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
+----------------------+   
| PydanticOutputParser |   
+----------------------+   
            *              
            *              
            *              
        +--------+         
        | Branch |         
        +--------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *       